# Probabilistic Forecasting - Electricity

This notebook demonstrates how to perform Data Analysis and Preparation Engineering with Amazon SageMaker Studio using AWS Glue Interactive Session.

Using this notebook, we can execute cells in order to read data, visualize, and perform transformations using PySpark with AWS Glue Interactice Session.

Let's start preparing our dataset.

**SageMaker Studio Kernel**: DataScience 3.0 - Python3

***

# Dataset

The data set (Electricity Price Forecasting) was downloaded from [Kaggle](https://www.kaggle.com/code/dimitriosroussis/electricity-price-forecasting-with-dnns-eda/data).
This dataset is using the past values of the electricity price as well as those of another features related to energy generation and weather conditions

# Step 1 - Import Modules

Here we’ll import some libraries and define some variables.

In [ ]:
import boto3
import sagemaker
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.spark.processing import PySparkProcessor

In [ ]:
sagemaker_client = boto3.client("sagemaker")
s3_client = boto3.client("s3")

Create a SageMaker Session and save the default region and the execution role in some Python variables

In [ ]:
sagemaker_session = sagemaker.Session()
region = boto3.session.Session().region_name
role = sagemaker.get_execution_role()

In [ ]:
bucket_name = sagemaker_session.default_bucket()

***

# Step 2 - Upload Python Scripts

In [ ]:
script_location = "electricity-forecasting/code/processing"

In [ ]:
# Download the
# clean the buckets first
s3_client.delete_object(Bucket=bucket_name, Key=script_location)

code_path = sagemaker_session.upload_data('./code', key_prefix=script_location)

code_path

***

# Step 2 - Run the processing job

By using [PySparkProcessor](https://sagemaker-examples.readthedocs.io/en/latest/sagemaker_processing/spark_distributed_data_processing/sagemaker-spark-processing.html), we can provide to the Amazon SageMaker Job the execution PySpark scripts in distributed data processing mode

In [ ]:
! pygmentize ./code/processing.py

## Global Parameters

In order to allow users to execute the SageMaker Processing Job locally, we are defining the variable `local_mode`. If you want to test the local mode capability, please put the variable to `True`

In [ ]:
local_mode = False

In [ ]:
processing_framework_version = "3.1"
processing_code = "electricity-forecasting/code/processing"
processing_input_files_path = "electricity-forecasting/data/input"
processing_instance_count = 2
processing_output_files_path = "electricity-forecasting/data/output"

if local_mode:
    processing_instance_type = "local"
else:
    processing_instance_type = "ml.m5.xlarge"

Define the `FrameworkProcessor` object

In [ ]:
processor = PySparkProcessor(
    framework_version=processing_framework_version,
    role=role,
    instance_count=processing_instance_count,
    instance_type=processing_instance_type,
    sagemaker_session=sagemaker_session
)

In [ ]:
run_args = processor.get_run_args(
        "processing.py",
        inputs=[
            ProcessingInput(
                input_name="input",
                source="s3://{}/{}".format(bucket_name, processing_input_files_path),
                destination="/opt/ml/processing/input"
            ),
            ProcessingInput(
                input_name="scripts",
                source="s3://{}/{}".format(bucket_name, processing_code),
                destination="/opt/ml/processing/input/code/scripts"
            )
        ],
        outputs=[
            ProcessingOutput(
                output_name="output",
                source="/opt/ml/processing/output",
                destination="s3://{}/{}".format(bucket_name, processing_output_files_path))
        ]
    )

In [ ]:
processor.run(
    "./code/processing.py",
    arguments=[
        "--copy_hdfs",
        "1",
        "--bucket_name",
        bucket_name,
        "--processing_input_files_path",
        processing_input_files_path,
        "--processing_output_files_path",
        processing_output_files_path
    ],
    inputs=run_args.inputs,
    outputs=run_args.outputs,
    spark_event_logs_s3_uri="s3://{}/electricity-forecasting/logs".format(bucket_name),
    wait=True
)